# 교차 검증 (Cross Validatain)
- 모델을 더욱 신뢰성 있게 평가하는 방법
- 데이터셋을 여러 개로 나누고, 각 부분이 한번씩 검증 데이터로 사용되도록 하는 방법
- 훈련-검증을 반복하면서 학습을 진행
- 과대적합 방지 효과

### K-fold

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 데이터 로드
from sklearn.datasets import load_iris

iris_input, iris_target = load_iris(return_X_y=True)
# np.unique(iris_target, return_counts=True) → (array([0, 1, 2]), array([50, 50, 50], dtype=int64)) # 각 클래스의 데이터 개수 50개 동일

(array([0, 1, 2]), array([50, 50, 50]))

In [ ]:
# 모델 생성
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

# 모델 생성
lr_clf = LogisticRegression()

# 교차검증 KFold 객체 생성 : 데이터를 K(n_splits)개의 묶음으로 나눠주는 역할
# - n_splits: 폴드의 개수, shuffle: 폴드로 나누기 전에 데이터를 섞을지 여부
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# 교차검증 수행, 학습과 검증을 반복하면서 평가
# k번 반복하면서 평가한 정확도를 저장할 배열
cv_accuracy = [] # 평가 점수를 계속 쌓을 수 있게 빈배열을 초기화

for train_index, val_index in kfold.split(iris_input):
    X_train, y_train = iris_input[train_index], iris_target[train_index] # 학습 데이터, fancy indexing
    X_val, y_val = iris_input[val_index], iris_target[val_index] # 검증 데이터

    print(np.unique(y_train, return_counts=True))
    print(np.unique(y_val, return_counts=True))
    print('---'*30)

    # oversampling 작동 원리
    # under sampling 작동 원리 

    # Sampling 편향을 일으킬 수도 있다.
    # 예) 클래스 0, 1, 2 데이터 개수가 50개씩 동일하지 않은 경우
    # iris데이터는 50개씩 동일한 학습 데이터


    # 모델 학습
    lr_clf.fit(X_train, y_train)              # 모델 학습
    y_pred = lr_clf.predict(X_val)            # 검증데이터로 예측한 결과
    acc_score = accuracy_score(y_val, y_pred) # 실제값, 예측값을 통한 정확도 계산
    cv_accuracy.append(acc_score)             # cv_accuracy 배열에 정확도 저장

print("훈련별 정확도:", cv_accuracy)
print("분류모델 정확도:", np.mean(cv_accuracy))


(array([0, 1, 2]), array([40, 41, 39]))
(array([0, 1, 2]), array([10,  9, 11]))
------------------------------------------------------------------------------------------
(array([0, 1, 2]), array([37, 40, 43]))
(array([0, 1, 2]), array([13, 10,  7]))
------------------------------------------------------------------------------------------
(array([0, 1, 2]), array([38, 40, 42]))
(array([0, 1, 2]), array([12, 10,  8]))
------------------------------------------------------------------------------------------
(array([0, 1, 2]), array([42, 40, 38]))
(array([0, 1, 2]), array([ 8, 10, 12]))
------------------------------------------------------------------------------------------
(array([0, 1, 2]), array([43, 39, 38]))
(array([0, 1, 2]), array([ 7, 11, 12]))
------------------------------------------------------------------------------------------
훈련별 정확도: [1.0, 1.0, 0.9333333333333333, 0.9666666666666667, 0.9666666666666667]
분류모델 정확도: 0.9733333333333334


In [9]:
for train_index, val_index in kfold.split(iris_input):
    print(f'train_index: {train_index},\n val_index: {val_index}\n'+'---'*30)

train_index: [  0   1   2   3   4   5   6   7   8  10  11  13  14  15  16  17  20  21
  22  23  24  25  27  28  32  33  34  35  37  38  39  40  41  42  43  44
  46  47  48  49  50  51  52  53  54  57  58  59  60  61  62  63  65  66
  67  70  71  72  74  75  77  79  80  81  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 105 106 107 109 111
 112 113 114 115 116 117 119 120 121 122 123 124 125 126 129 130 133 134
 135 136 137 138 139 140 142 144 146 147 148 149],
 val_index: [  9  12  18  19  26  29  30  31  36  45  55  56  64  68  69  73  76  78
  82 104 108 110 118 127 128 131 132 141 143 145]
------------------------------------------------------------------------------------------
train_index: [  1   2   3   5   6   7   8   9  12  13  14  17  18  19  20  21  23  24
  25  26  29  30  31  33  34  35  36  37  38  39  41  43  45  46  47  48
  49  50  52  53  54  55  56  57  58  59  61  62  63  64  68  69  70  71
  72  73  74  76  77  78  79  80  82  83

In [18]:
iris_input.shape, iris_target.shape

((150, 4), (150,))

### **StratifiedKFold**

In [ ]:
# StratifiedKFold : 분류 모델에서 주로 사용
# 데이터 분할 시 각 클래스의 데이터 비율을 유지하는 방법

from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 모델 생성
lr_clf = LogisticRegression()

# 교차검증 StratifiedKFold 객체 생성 : 데이터를 K(n_splits)개의 묶음으로 나눠주는 역할, 각 폴드에서 각 클래스의 데이터 비율을 유지
# - n_splits: 폴드의 개수, shuffle: 폴드로 나누기 전에 데이터를 섞을지 여부
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# k번 반복하면서 평가한 정확도를 저장할 배열
cv_accuracy = [] # 평가 점수를 계속 쌓을 수 있게 빈배열을 초기화
                                                                               # target 데이터의 label을 전달해야 비율을 유지할 수 있음
for train_index, val_index in stratified_kfold.split(iris_input, iris_target): # iris_input, iris_target 데이터를 넣어줘야 함
    X_train, y_train = iris_input[train_index], iris_target[train_index]       # 학습 데이터, fancy indexing
    X_val, y_val = iris_input[val_index], iris_target[val_index]               # 검증 데이터

    print(np.unique(y_train, return_counts=True))
    print(np.unique(y_val, return_counts=True))
    print('---'*30)

    # KFold 는 Sampling 편향을 일으킬 수도 있다. → StratifiedKFold 는 비율을 유지하므로 편향을 줄일 수 있다.
    # 예) 클래스 0, 1, 2 데이터 개수가 50개씩 동일하지 않은 경우
    # iris데이터는 50개씩 동일한 학습 데이터


    # 모델 학습
    lr_clf.fit(X_train, y_train)              # 모델 학습
    y_pred = lr_clf.predict(X_val)            # 검증데이터로 예측한 결과
    acc_score = accuracy_score(y_val, y_pred) # 실제값, 예측값을 통한 정확도 계산
    cv_accuracy.append(acc_score)             # cv_accuracy 배열에 정확도 저장

print("훈련별 정확도:", cv_accuracy)
print("분류모델 정확도:", np.mean(cv_accuracy))


(array([0, 1, 2]), array([40, 40, 40]))
(array([0, 1, 2]), array([10, 10, 10]))
------------------------------------------------------------------------------------------
(array([0, 1, 2]), array([40, 40, 40]))
(array([0, 1, 2]), array([10, 10, 10]))
------------------------------------------------------------------------------------------
(array([0, 1, 2]), array([40, 40, 40]))
(array([0, 1, 2]), array([10, 10, 10]))
------------------------------------------------------------------------------------------
(array([0, 1, 2]), array([40, 40, 40]))
(array([0, 1, 2]), array([10, 10, 10]))
------------------------------------------------------------------------------------------
(array([0, 1, 2]), array([40, 40, 40]))
(array([0, 1, 2]), array([10, 10, 10]))
------------------------------------------------------------------------------------------
훈련별 정확도: [1.0, 0.9666666666666667, 0.9333333333333333, 1.0, 0.9333333333333333]
분류모델 정확도: 0.9666666666666668


c:\Users\Playdata\anaconda3\envs\ml_env\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Playdata\anaconda3\envs\ml_env\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/pre

### cross_val_score
- 교차검증을 통해 모델 성능을 평가하는 함수
- 내부적으로 지정한 횟수만큼 학습/검증을 나누어 반복 처리

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html

In [ ]:
from sklearn.model_selection import cross_val_score

lr_clf = LogisticRegression(max_iter=1000)

# 첫번째 인자 : 모델
# 두번째 인자 : 특성 데이터
# 세번째 인자 : 라벨 데이터
# 네번째 인자 : 반복횟수(내부적으로 KFold 객체를 사용) → 데이터를 알아서 나눠서, 학습을 하고 평가를 한다. (나눠지는 기준 : KFold 객체)
    # cv : 반복 횟수 → int, BaseCrossValidator 객체, Iterable[int] → 전달 가능 → StratifiedKFold 객체 전달 가능
        
        # For int/None inputs, if the estimator is a classifier and y is either binary or multiclass, StratifiedKFold is used.
        # In all other cases, KFold is used. These splitters are instantiated with shuffle=False so the splits will be the same across calls.
        # Refer User Guide for the various cross-validation strategies that can be used here.

        # 분류 문제에서는 라벨 데이터가 이진 또는 다중 클래스인 경우 StratifiedKFold 사용
        # 입력 의 None경우, 추정기가 분류기이고 y이진 또는 다중 클래스이면 가 StratifiedKFold사용됩니다.
        # 다른 모든 경우에는 KFold가 사용됩니다. 이러한 분할기는 로 인스턴스화되므로 shuffle=False호출 전체에서 분할이 동일합니다.


# scoring : 평가 지표(정확도, 정밀도, 재현율, F1-score, 등) → 문자열, callable, None → 전달 가능
    # accuracy(default), precision, recall, f1, roc_auc, 등
    # 전달되지 않으면 모델의 평가 지표를 사용
    # 전달되는 값은 모델의 평가 지표를 사용
# 반환값 : 반복할 훈련별 검증 점수 '배열' 
    # 각 반복에서 얻은 정확도가 배열로 반환된다.
# 예) 5개의 폴드로 나누어 5번 반복 평가 → 5개의 정확도 배열 반환

scores = cross_val_score(lr_clf, iris_input, iris_target, cv=5, scoring='accuracy') # 모델, 데이터, 반복 횟수, 평가 지표(정확도)

print('훈련별 정확도:', scores)
print('모델 평균 정확도:', np.mean(scores))

훈련별 정확도: [0.96666667 1.         0.93333333 0.96666667 1.        ]
모델 평균 정확도: 0.9733333333333334


In [ ]:
from sklearn.model_selection import cross_validate


# 모델 객체
lr_clf = LogisticRegression(max_iter=1000)

# cross_validate 함수는 여러 평가 지표를 한번에 사용할 수 있다. → 다중 평가지표
# f1_macro : 각 클래스의 평균 f1-score, 이진 분류 모델에서 사용
# return_train_score : 훈련 데이터로 평가한 점수도 반환

scores = cross_validate(lr_clf, iris_input, iris_target, cv=5, scoring=['accuracy', 'f1_macro'], return_train_score=True)

scores

{'fit_time': array([0.04074907, 0.01844144, 0.        , 0.        , 0.00614572]),
 'score_time': array([0.00201201, 0.00050521, 0.01518726, 0.01505351, 0.00989747]),
 'test_accuracy': array([0.96666667, 1.        , 0.93333333, 0.96666667, 1.        ]),
 'train_accuracy': array([0.96666667, 0.96666667, 0.98333333, 0.98333333, 0.975     ]),
 'test_f1_macro': array([0.96658312, 1.        , 0.93265993, 0.96658312, 1.        ]),
 'train_f1_macro': array([0.96664582, 0.96664582, 0.98333333, 0.98332291, 0.97499609])}

In [ ]:
lr_clf.predict(iris_input)
# NotFittedError: This LogisticRegression instance is not fitted yet.

# cross_val_score, cross_validate 는 모델 학습을 하지 않는다.
# 내부적으로는 학습을 해서 평가를 하는 것이지만, 실제 학습된 모델을 반환하지 않는다.

NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.